# 

# Automated Optical Inspection at Variass
Help Variass in predicting the defect rates of assembled products based on their specifications

predict the quality (rate of defects per defect type) based on a given Valor code (or list of Valor codes cq the bill of materials of a completely new product)

## Step 1: import python modules and read data

In [1]:
import pandas as pd
import numpy as np
import os
import sys

## Preparing data
Read excel AOI, order list and VPL (taking around 31s) and converting it to csv for better performance

In [38]:
file_path_order = "data/initial/Order picklists.xlsx"
file_path_vpl = "data/initial/VPL list.xlsx"
pd.read_excel(file_path_order).to_csv("data/orders.csv", index=False)
pd.read_excel(file_path_vpl).to_csv("data/vpl_codes.csv", index=False)

Read the new csv files as three new dataframes 

In [39]:

df_aoi_defects = pd.read_csv("data/initial/AOI defects last year.csv")
df_orders = pd.read_csv("data/orders.csv")
df_vpl = pd.read_csv("data/vpl_codes.csv")

In [41]:
df_aoi_defects.head()
df_aoi_defects.columns

,overkill,defecttypestring,moduleid,refid,frameid,windowid,isimportantwindow,partnumber,partid,windowtype,...,framerecty4,defectrectx1,defectrectx2,defectrectx3,defectrectx4,defectrecty1,defectrecty2,defectrecty3,defectrecty4,id
0,0,Missing,0,SC68-P,33,2,0,UHW-00003,5532728F-188C-43fc-97D8-C9FBBE46CF3B,1,...,167846.0,499243.0,500321.0,500321.0,499243.0,149534.0,149534.0,150579.0,150579.0,1
1,0,Missing,0,U1-KD,25,0,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,1,...,149048.0,428648.0,436050.0,436050.0,428648.0,114934.0,114934.0,116459.0,116459.0,2
2,0,Missing,0,U1-KD,25,1,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,5,...,149048.0,436675.0,448295.0,448295.0,436675.0,117988.0,117988.0,119084.0,119084.0,3
3,0,Missing,0,U1-KD,25,2,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,5,...,149048.0,424899.0,436519.0,436519.0,424899.0,117991.0,117991.0,119087.0,119087.0,4
4,0,Missing,0,U1-KD,25,3,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,1,...,149048.0,437847.0,440247.0,440247.0,437847.0,120726.0,120726.0,123247.0,123247.0,5


In [42]:
df_orders.head()

,Order,part number,total amount
0,1036933,CAP-00693,504.0
1,1036933,CMO-00180,252.0
2,1036933,CZZ-00214,252.0
3,1036933,CZZ-00215,252.0
4,1036933,CZZ-00216,252.0


In [43]:
df_vpl.head()

,partnumber,mpn,VPLpackage,Material type,Position type,Package type,Lead type,Pitch,Subtype
0,VVH-00021,TSM4ZJ202KR10,PDSO-J3/XX-L48W46T37,P,D,SO,J3,X,X
1,INS-00410,LF353M/NOPB,PDSO-G8/FX-L49W39T18,P,D,SO,G8,F,X
2,VWO-00003,82541300,XDXD-R2/XR-L32W16T15,X,D,XD,R2,X,R
3,FMU-00181,NFM3DCC102R1H3L,XXXD-R3/XL-L32W13T7,X,X,XD,R3,X,L
4,KKT-00218,T495X337K010ATE100,PDSO-C2/XX-L73W43T40,P,D,SO,C2,X,X
